In [1]:
#Takes rectangular salary schedules (of any dimension), creates a separate row for each unit amount, and combines into a single master dataset for all colleges

import pandas as pd
import numpy as np
import os

In [2]:
#If not already in place, creates "prepped_schedules" folder and deletes previous prepped schedules

def prep_structure():
    if os.path.isdir('../prepped_schedules') == False:
        os.mkdir('../prepped_schedules')
    for dirname, _, filenames in os.walk('../prepped_schedules'):
        for filename in filenames:
                filename = '../prepped_schedules/' + filename
                os.remove(filename)

In [3]:
#load salary schedules as dframes under name "df_fake_XXXCCD"

def load_sched():
    global districts
    districts = []
    for dirname, _, filenames in os.walk('../transposed_schedules'):
        for filename in filenames:
            if filename.endswith('.csv'):
                    globals()['df_{}'.format(filename[0:-4])] = pd.read_csv('../transposed_schedules/' + filename)
                    #print(filename + " has been imported as df_"+filename[0:-4])
                    if filename[0:-4] not in districts :
                        districts.append(filename[0:-4])
    globals()['df_long'] = pd.read_csv('../longevity_data/longevity.csv')

In [4]:
#Insure that units measurements are integers and impute 500 units to the PhD class    

def phd_to_num():
    for dist in districts:
        z = eval('df_' + dist)
        z['m_units'] = z.apply(lambda x: int(round(float(x['m_units']))) if x['m_units']!='phd' else int(500), axis = 1)
        z['b_units'] = z.apply(lambda x: int(round(float(x['b_units']))) if x['b_units']!='phd' else int(500), axis = 1)

In [5]:
#Identify the maximum step value for each schedule

def max_step():
    global years
    years = 0
    for dist in districts:
        z = eval('df_' + dist)
        num_cols = len(z.columns)
        if num_cols -3 >years:
            years = num_cols-2
    print('The highest step is ' + str(years))

In [6]:
#create separate row for each distinct unit amount above master's (up to 96) and bachelor's (up to 146) degrees

def merge_sched():
    print('Merging all schedules together; this might take a few minutes...')
    global data
    global final_data
    
    data = {}
    
    df1 = pd.DataFrame()
    
    for dist in districts: #running through each individual schedule

        df = pd.DataFrame(columns = ['b_units', 'm_units'])

        for i in range(147):
            df = df.append({'b_units' : i, 'm_units': 499}, ignore_index = True)

        for i in range(97):
            df = df.append({'b_units' : 499, 'm_units': i}, ignore_index = True)

        z = eval('df_' + dist)

        t = len(z.columns) 

        for i in range(1, t-1):
            df[str(i)]=0 #create columns for each step
        y = len(z) #identify how many salary classes there are
        for j in range(1,t-1): #create new rows for each step
            k = str(j)
            for i in range(y-1): #loops through each class
                if i < (y-2): #if class cutoff is not the last one, finds all unit values at or above this cutoff and below the next one and populates the column with those salaries

                    df[k] = df.apply(lambda x: z[k][i] if (x[k]==0) & (z['m_units'][i] <= x['m_units']) 
                                     & (x['m_units'] < z['m_units'][i+1]) else (z[k][i] if (x[k]==0) & (z['b_units'][i] <= x['b_units']) 
                                     & (x['b_units'] < z['b_units'][i+1]) else x[k]), axis = 1)

                else: #if class cutoff is the last one, finds all unit values at or above the cutoff to populate the column
                    df[k] = df.apply(lambda x: z[k][i] if (x[k]==0) & (z['m_units'][i] <= x['m_units']) 
                                     else (z[k][i] if (x[k]==0) & (z['b_units'][i] <= x['b_units']) 
                                     else x[k]), axis = 1)   

        new_row = z.iloc[[y-1]]

        df = df.append(new_row, ignore_index=True)
            
        if t-2 < years: #if this schedule has fewer steps than the max number of steps across all schedules, fills in remaining steps with max salary for class
            for i in range(t-1, years+1):
                df[str(i)]= df[str(t-2)]
        
        
        df['dist_code']= dist #creates district column
        
         

        #Reshape data set by creating unique rows for each combination of units, step, and and district 
        vals = df.columns.to_list()
        
        rem_vals = ['m_units','b_units', 'dist_code']
        
        for col in rem_vals:
            vals.remove(col)
        
        df = pd.melt(df, id_vars = ['m_units', 'b_units', 'dist_code'], value_vars = vals, value_name = 'salary', var_name = 'step')


        if dist in df_long['dist code'].to_list():
            for i in range(26):
                mult = df_long[str(i)][df_long['dist code'] == dist]
                df[str(i)] = df.apply(lambda x: mult, axis = 1)

            vals = df.columns.to_list()
            rem_vals = ['m_units','b_units', 'dist_code', 'salary', 'step']
            for col in rem_vals:
                vals.remove(col)

            df = pd.melt(df, id_vars = ['m_units','b_units', 'dist_code', 'salary', 'step'], value_vars = vals, value_name = 'long_factor', var_name = 'dist_service')

            df['salary'] = df.apply(lambda x: (x['salary'])*(x['long_factor']), axis = 1)
        
            df.drop(columns = ['long_factor'], inplace = True)
        else:
            for i in range(26):
                df['dist_service']=-1
        
        df_name = dist + 'a'

        data[df_name]=df #save each separate district's melted dframe in a dictionary
        
        df1 = df1.append(df) #append each district's dframe to the master merged dframe

        final_data = df1
        

In [7]:
#save the individual prepped schedules and the master merged schedules

def save_prepped_sched():
    for key, value in data.items():
        data[key].to_csv('../prepped_schedules/' + key + '.csv', index = False)
    
    final_data.to_csv('../prepped_schedules/comp_data.csv', index = False)

In [8]:
prep_structure()

In [9]:
load_sched()

In [10]:
phd_to_num()

In [11]:
max_step()

The highest step is 50


In [12]:
merge_sched()

Merging all schedules together; this might take a few minutes...


In [13]:
save_prepped_sched()